In [2]:
import re
import sys
from pyspark import SparkConf, SparkContext
from itertools import combinations
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile('soc-short.txt')
user_friend = lines.map(lambda l:tuple(l.split('\t')))
user_ratingprod = user_friend.flatMap(lambda x:[((x[0],i),-1000) for i in x[1].split(',')])
u_friend = user_friend.map(lambda x:(x[1].split(',')))
user_ratingprod2 = u_friend.flatMap(lambda w:[(i,1) for i in combinations(w,2)])
#user_ratingprod2 = user_friend.flatMap(lambda x: (lambda j:[[((j[u],j[i]),1) for i in range(u+1,len(j))]for u in range(0,len(j))])(x[1].split(',')))
#user_ratingprod2 = user_ratingprod2.flatMap(lambda xs: [x for x in xs])
unio = user_ratingprod.union(user_ratingprod2).reduceByKey(lambda n1,n2:n1+n2)
unio = unio.map(lambda x: (x[0][0],x[0],x[1]))
uni_list = unio.keys().distinct().collect()
charRDD = []
for i in uni_list:
    char = i
    user = unio.filter(lambda x: x[0]==char)
    top_10 = user.filter(lambda u:u[2] >= 0).takeOrdered(10, key=lambda x: -x[2])
    if len(top_10) == 10:
        charRDD.append(top_10)
    else:
        user_friend = user.map(lambda x:x[1][1]).collect()
        
        for j in uni_list:
            if j not in user_friend:
                top_10.append((char,(char,j),"random friend"))
                if len(top_10) == 10:
                    charRDD.append(top_10)
                    break
charRDD = sc.parallelize(charRDD)
#charRDD.saveAsTextFile(sys.argv[2])
#sc.stop()
charRDD.collect()

[[('4', ('4', '85'), 2),
  ('4', ('4', '83'), 2),
  ('4', ('4', '38'), 2),
  ('4', ('4', '46'), 2),
  ('4', ('4', '9'), 1),
  ('4', ('4', '17'), 1),
  ('4', ('4', '19'), 1),
  ('4', ('4', '21'), 1),
  ('4', ('4', '22'), 1),
  ('4', ('4', '29'), 1)],
 [('10', ('10', '29'), 2),
  ('10', ('10', '85'), 2),
  ('10', ('10', '83'), 2),
  ('10', ('10', '38'), 2),
  ('10', ('10', '41'), 2),
  ('10', ('10', '55'), 2),
  ('10', ('10', '89'), 2),
  ('10', ('10', '14'), 1),
  ('10', ('10', '17'), 1),
  ('10', ('10', '19'), 1)],
 [('12', ('12', '14'), 1),
  ('12', ('12', '17'), 1),
  ('12', ('12', '19'), 1),
  ('12', ('12', '21'), 1),
  ('12', ('12', '22'), 1),
  ('12', ('12', '33'), 1),
  ('12', ('12', '34'), 1),
  ('12', ('12', '45'), 1),
  ('12', ('12', '48'), 1),
  ('12', ('12', '63'), 1)],
 [('29418', ('29418', '4'), 'random friend'),
  ('29418', ('29418', '10'), 'random friend'),
  ('29418', ('29418', '12'), 'random friend'),
  ('29418', ('29418', '29418'), 'random friend'),
  ('29418', ('2941